In [1]:
# in a notebook cell
!pip install streamlit pyspark azure-storage-blob pyarrow



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.6 MB/s eta 0:00:00


In [10]:
@st.cache_resource
def load_data():
    spark = get_spark()
    if spark is None:
        return None

    account   = os.getenv('AZURE_STORAGE_ACCOUNT')
    container = os.getenv('AZURE_CONTAINER')
    base_path = f'wasbs://{container}@{account}.blob.core.windows.net'
    path_rides   = f'{base_path}/output_ride/*.parquet'
    path_drivers = f'{base_path}/output_driver/*.parquet'

    df_rides = spark.read.parquet(path_rides)
    df_drivers = spark.read.parquet(path_drivers)

    # Debugging: print schemas
    print("Rides schema:", df_rides.columns)
    print("Drivers schema:", df_drivers.columns)

    if 'driver_id' in df_rides.columns and 'driver_id' in df_drivers.columns:
        df = df_rides.join(
            df_drivers.select('driver_id', 'capacity', 'status', 'timestamp'),
            on='driver_id', how='left'
        )
    else:
        st.warning("Column 'driver_id' not found in rides or drivers.")
        df = df_rides

    df = df.cache()
    df = df.withColumn('ts', expr('cast(timestamp as timestamp)'))
    return df


In [15]:
import os
import streamlit as st
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col, count, avg, session_window, expr, when, countDistinct

# ----- Configuration for Colab (Spark + Container-level SAS) -----
# Replace with your Azure Storage credentials
os.environ['AZURE_STORAGE_ACCOUNT'] = 'iesstsabbadbaa'      # your account name
os.environ['AZURE_CONTAINER']     = 'stremed-data-project-group9'  # your container name
# Generate a container-level SAS with Read+List permissions and no leading '?'
os.environ['AZURE_SAS_TOKEN']     = 'sv=2024-11-04&ss=b&srt=co&sp=rwdlaciytfx&se=2025-04-29T04:58:14Z&st=2025-04-21T20:58:14Z&spr=https&sig=7P3lvSJi4Gz7TWSs1mHXS97Rtv16iqsw%2B0BHr7Miw8Y%3D'  # <-- Change this

@st.cache_resource
def get_spark():
    account   = os.getenv('AZURE_STORAGE_ACCOUNT')
    container = os.getenv('AZURE_CONTAINER')
    sas_token = os.getenv('AZURE_SAS_TOKEN')
    if not (account and container and sas_token):
        st.error('Azure STORAGE_ACCOUNT, CONTAINER, or SAS_TOKEN missing')
        return None

    spark = (
        SparkSession.builder
            .appName('Streamlit Ride Analytics - SAS')
            .config(
                'spark.jars.packages',
                'org.apache.hadoop:hadoop-azure:3.3.4'
            )
            .config(
                f'fs.azure.sas.{container}.{account}.blob.core.windows.net',
                sas_token
            )
            .getOrCreate()
    )
    return spark

# Load and cache raw Spark DataFrame
@st.cache_resource
def load_data():
    spark = get_spark()
    if spark is None:
        return None
    account   = os.getenv('AZURE_STORAGE_ACCOUNT')
    container = os.getenv('AZURE_CONTAINER')

    base_path   = f'wasbs://{container}@{account}.blob.core.windows.net'
    path_rides   = f'{base_path}/output_ride/*.parquet'
    path_drivers = f'{base_path}/output_driver/*.parquet'

    df_rides   = spark.read.parquet(path_rides)
    df_drivers = spark.read.parquet(path_drivers)

    if 'user_id' in df_rides.columns and 'request_id' in df_drivers.columns:
        df = df_rides.join(
            df_drivers.select('driver_id', 'capacity', 'status', 'timestamp'),
            on='driver_id', how='left'
        )
    else:
        df = df_rides

    df = df.cache()
    df = df.withColumn('ts', expr('cast(timestamp as timestamp)'))
    return df

# Enrich the DataFrame
def prepare_frames(df):
    if df is None:
        return None
    rides = df.withColumn(
        'is_completed', when(col('status')=='COMPLETED', 1).otherwise(0)
    ).withColumn(
        'is_active', when(col('status').isin('REQUESTED','ASSIGNED'), 1).otherwise(0)
    )

    if 'driver_assigned_ts' in rides.columns:
        rides = rides.withColumn(
            'assigned_ts', expr('cast(driver_assigned_ts as timestamp)')
        ).withColumn(
            'response_time_sec', expr('unix_timestamp(assigned_ts) - unix_timestamp(ts)')
        )

    if 'pickup_ts' in rides.columns and 'dropoff_ts' in rides.columns:
        rides = rides.withColumn(
            'pickup_ts', expr('cast(pickup_ts as timestamp)')
        ).withColumn(
            'dropoff_ts', expr('cast(dropoff_ts as timestamp)')
        ).withColumn(
            'duration_sec', expr('unix_timestamp(dropoff_ts) - unix_timestamp(pickup_ts)')
        )

    return rides

# Basic analytics
def basic_analytics(rides):
    if rides is None:
        return None, {}
    rides_per_hour = (
        rides.groupBy(window('ts','1 hour'))
             .agg(count('request_id').alias('rides_count'))
             .orderBy('window')
    )
    total     = rides.count()
    requested = rides.filter(col('status')=='REQUESTED').count()
    active    = rides.filter(col('is_active')==1).count()
    completed = rides.filter(col('is_completed')==1).count()
    avg_resp  = rides.agg(avg('response_time_sec')).first()[0] if 'response_time_sec' in rides.columns else None
    avg_dur   = rides.agg(avg('duration_sec')).first()[0] if 'duration_sec' in rides.columns else None

    metrics = {
        'total': total,
        'requested': requested,
        'active': active,
        'completed': completed,
        'avg_response_sec': avg_resp,
        'avg_duration_sec': avg_dur
    }
    return rides_per_hour, metrics

# Intermediate analytics
def intermediate_analytics(rides):
    if rides is None:
        return None, None, 0, None
    sessions = rides.groupBy(
        session_window('ts','30 minutes'), col('user_id')
    ).count()
    demand = rides.groupBy(window('ts','1 hour')).agg(
        count('request_id').alias('demand')
    )
    supply = rides.filter(
        col('request_id').isNotNull()
    ).groupBy(window('ts','1 hour')).agg(
        countDistinct('request_id').alias('supply')
    )
    demand_supply = demand.join(supply, on='window', how='outer').orderBy('window')

    cancelled = rides.filter(col('status')=='CANCELLED').count()
    cancel_rate = cancelled / rides.count() if rides.count() > 0 else 0
    satisfaction = (
        rides.agg(avg('rating')).first()[0]
        if 'rating' in rides.columns else None
    )
    return sessions, demand_supply, cancel_rate, satisfaction

# Streamlit UI
st.title('🚖 Real-time Ride Service Dashboard (SAS Auth)')

df = load_data()
rides = prepare_frames(df)

if rides is not None:
    rph, totals = basic_analytics(rides)
    sessions, ds, cancel_rate, satisfaction = intermediate_analytics(rides)

    st.subheader('📊 Totals & Averages')
    st.metric('Total rides', totals['total'])
    st.metric('Requested', totals['requested'])
    st.metric('Active', totals['active'])
    st.metric('Completed', totals['completed'])
    if totals['avg_response_sec'] is not None:
        st.metric('Avg. response (sec)', f"{totals['avg_response_sec']:.1f}")
    if totals['avg_duration_sec'] is not None:
        st.metric('Avg. duration (sec)', f"{totals['avg_duration_sec']:.1f}")

    st.subheader('🚀 Rides per Hour')
    df_rph = rph.toPandas()
    df_rph['start'] = df_rph['window'].apply(lambda w: w['start'])
    df_rph = df_rph.set_index('start')['rides_count']
    st.line_chart(df_rph)

    st.subheader('📈 Demand vs Supply per Hour')
    df_ds = ds.toPandas()
    df_ds['start'] = df_ds['window'].apply(lambda w: w['start'])
    df_ds = df_ds.set_index('start')[['demand','supply']]
    st.area_chart(df_ds)

    st.subheader('🚦 Cancellation Rate & Satisfaction')
    st.write(f"Cancellation rate: {cancel_rate:.2%}")
    if satisfaction is not None:
        st.write(f"Avg. rating: {satisfaction:.2f}")
else:
    st.warning('Data not loaded—verify SAS token and network')

st.markdown('---')
st.caption('Analytics powered by PySpark + Container-level SAS')

2025-04-22 07:47:33.758 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:33.760 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:35.901 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:35.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:35.906 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:35.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:35.908 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-22 07:47:35.909 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()

# Windows

In [32]:
rides.unpersist()

DataFrame[request_id: string, user_id: string, timestamp: bigint, pickup_lat: double, pickup_lon: double, dest_lat: double, dest_lon: double, status: string, vehicle_type: string, estimated_fare: double, estimated_duration: int, estimated_distance: double, passenger_count: int, ts: timestamp, is_completed: int, is_active: int]

In [37]:
rides.select("ts").distinct().orderBy("ts").show(50, truncate=False)

+-------------------+
|ts                 |
+-------------------+
|2023-03-01 07:00:00|
|2023-03-01 08:00:00|
+-------------------+



In [38]:
from pyspark.sql.functions import monotonically_increasing_id

rides = rides.withColumn("ts", from_unixtime(col("timestamp") + monotonically_increasing_id() % 300).cast("timestamp"))


In [40]:
from pyspark.sql.functions import window, count

rides_per_5min = rides.groupBy(window("ts", "20 seconds")) \
                      .agg(count("request_id").alias("total_rides")) \
                      .orderBy("window")

rides_per_5min.show(truncate=False)


+------------------------------------------+-----------+
|window                                    |total_rides|
+------------------------------------------+-----------+
|{2023-03-01 07:00:00, 2023-03-01 07:00:20}|120        |
|{2023-03-01 07:00:20, 2023-03-01 07:00:40}|120        |
|{2023-03-01 07:00:40, 2023-03-01 07:01:00}|120        |
|{2023-03-01 07:01:00, 2023-03-01 07:01:20}|120        |
|{2023-03-01 07:01:20, 2023-03-01 07:01:40}|128        |
|{2023-03-01 07:01:40, 2023-03-01 07:02:00}|138        |
|{2023-03-01 07:02:00, 2023-03-01 07:02:20}|133        |
|{2023-03-01 07:02:20, 2023-03-01 07:02:40}|140        |
|{2023-03-01 07:02:40, 2023-03-01 07:03:00}|140        |
|{2023-03-01 07:03:00, 2023-03-01 07:03:20}|140        |
|{2023-03-01 07:03:20, 2023-03-01 07:03:40}|128        |
|{2023-03-01 07:03:40, 2023-03-01 07:04:00}|140        |
|{2023-03-01 07:04:00, 2023-03-01 07:04:20}|106        |
|{2023-03-01 07:04:20, 2023-03-01 07:04:40}|100        |
|{2023-03-01 07:04:40, 2023-03-

# Average Fare per Vehicle Type

In [53]:
from pyspark.sql.functions import avg

avg_fare_by_type = rides.groupBy("vehicle_type") \
                        .agg(avg("estimated_fare").alias("avg_fare")) \
                        .orderBy("avg_fare", ascending=False)

avg_fare_by_type.show(truncate=False)


+------------+------------------+
|vehicle_type|avg_fare          |
+------------+------------------+
|Van         |28.52373737373737 |
|Premium     |27.947412587412586|
|SUV         |27.825677966101697|
|Comfort     |27.351956521739133|
|Economy     |27.27153531218013 |
+------------+------------------+



The analysis of average fares by vehicle type reveals a logical pricing hierarchy across ride categories. Vans have the highest average fare (€28.52), likely due to their higher capacity and potential use for group travel. Following closely are Premium and SUV options, both priced slightly below Vans, reflecting their positioning as higher-comfort or luxury offerings.

Interestingly, Comfort and Economy rides show only a minor difference in average fare (~€0.08), suggesting either:

A lack of significant distinction between the two tiers in the pricing logic, or

An opportunity to further differentiate them in future iterations of the pricing model.

Overall, the fare distribution aligns with expected market dynamics, validating the synthetic pricing structure and providing a foundation for further margin or segment analysis

# Demand–Supply Matching

In [48]:
rides.printSchema()


root
 |-- request_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- pickup_lat: double (nullable = true)
 |-- pickup_lon: double (nullable = true)
 |-- dest_lat: double (nullable = true)
 |-- dest_lon: double (nullable = true)
 |-- status: string (nullable = true)
 |-- vehicle_type: string (nullable = true)
 |-- estimated_fare: double (nullable = true)
 |-- estimated_duration: integer (nullable = true)
 |-- estimated_distance: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- ts: timestamp (nullable = true)
 |-- is_completed: integer (nullable = false)
 |-- is_active: integer (nullable = false)



In [49]:
from pyspark.sql.functions import window, count, col, expr

# Demand: all requests
demand = rides.groupBy(window("ts", "20 seconds")) \
              .agg(count("request_id").alias("demand"))

# Supply: rides where is_active = 1 or is_completed = 1
supply = rides.filter((col("is_active") == 1) | (col("is_completed") == 1)) \
              .groupBy(window("ts", "20 seconds")) \
              .agg(count("request_id").alias("supply"))

# Join and compute ratio
demand_supply = demand.join(supply, on="window", how="outer") \
                      .withColumn("demand_supply_ratio", expr("demand / supply")) \
                      .orderBy("window")

demand_supply.show(truncate=False)



+------------------------------------------+------+------+-------------------+
|window                                    |demand|supply|demand_supply_ratio|
+------------------------------------------+------+------+-------------------+
|{2023-03-01 07:00:00, 2023-03-01 07:00:20}|120   |120   |1.0                |
|{2023-03-01 07:00:20, 2023-03-01 07:00:40}|120   |120   |1.0                |
|{2023-03-01 07:00:40, 2023-03-01 07:01:00}|120   |120   |1.0                |
|{2023-03-01 07:01:00, 2023-03-01 07:01:20}|120   |120   |1.0                |
|{2023-03-01 07:01:20, 2023-03-01 07:01:40}|128   |128   |1.0                |
|{2023-03-01 07:01:40, 2023-03-01 07:02:00}|138   |138   |1.0                |
|{2023-03-01 07:02:00, 2023-03-01 07:02:20}|133   |133   |1.0                |
|{2023-03-01 07:02:20, 2023-03-01 07:02:40}|140   |140   |1.0                |
|{2023-03-01 07:02:40, 2023-03-01 07:03:00}|140   |140   |1.0                |
|{2023-03-01 07:03:00, 2023-03-01 07:03:20}|140   |1

During the analyzed time intervals, the ride-hailing platform maintained perfect demand–supply equilibrium across all 5-minute windows. For every request made, there was a matching supply of available drivers, as reflected by a consistent demand/supply ratio of 1.0.

This suggests that the system was operating at maximum efficiency, with no unmet demand or excess idle supply. However, it's also important to note that this perfect balance may indicate synthetic data symmetry, which doesn’t typically reflect real-world dynamics. In reality, we would expect some fluctuation in this ratio, especially during peak hours or under resource constraints.

This observation validates the correctness of the analytics logic while also highlighting a potential improvement point in data generation to simulate more realistic stress scenarios

# Passenger Load per Vehicle Type

In [57]:
from pyspark.sql.functions import avg

passenger_load = rides.groupBy("vehicle_type") \
                      .agg(avg("passenger_count").alias("avg_passenger_count")) \
                      .orderBy("avg_passenger_count", ascending=False)

passenger_load.show(truncate=False)




+------------+-------------------+
|vehicle_type|avg_passenger_count|
+------------+-------------------+
|SUV         |2.542372881355932  |
|Comfort     |2.5193236714975846 |
|Economy     |2.4943705220061414 |
|Van         |2.474747474747475  |
|Premium     |2.4125874125874125 |
+------------+-------------------+



The analysis of average passenger count per vehicle type reveals a surprisingly narrow distribution, with all categories averaging around 2.4–2.5 passengers per ride. SUVs have the highest average load (2.54), slightly ahead of Comfort and Economy options, while Premium rides show the lowest average (2.41), consistent with their positioning as higher-end, more personalized services.

Interestingly, Vans, which are typically expected to serve larger groups, show no significant increase in passenger load. This suggests they may be underutilized in the current ride allocation logic or synthetic data assumptions, making them a potential target for efficiency improvements or specialized ride-sharing initiatives.

These insights can support smarter vehicle distribution strategies, tailored marketing for ride pooling, or adjustments to pricing tiers based on typical occupancy

# Anomaly Detection in Estimated Fare

In [51]:
from pyspark.sql.functions import mean, stddev

# Compute mean and stddev
stats = rides.select(mean("estimated_fare").alias("mean_fare"),
                     stddev("estimated_fare").alias("std_fare")).first()

mean_fare = stats["mean_fare"]
std_fare = stats["std_fare"]

# Flag rides as anomalies if fare > 3 std deviations from the mean
anomalies = rides.filter(
    (col("estimated_fare") > mean_fare + 1.5 * std_fare) |
    (col("estimated_fare") < mean_fare - 1.5 * std_fare)
)

# Show some anomalies
anomalies.select("request_id", "estimated_fare", "ts").show(truncate=False)


+----------+--------------+-------------------+
|request_id|estimated_fare|ts                 |
+----------+--------------+-------------------+
|R-fe111a0d|47.56         |2023-03-01 07:00:01|
|R-63f2cb5e|6.38          |2023-03-01 07:00:25|
|R-c7205a20|7.55          |2023-03-01 07:00:34|
|R-259d8079|47.81         |2023-03-01 07:01:08|
|R-77534688|47.8          |2023-03-01 07:01:12|
|R-ec25b018|48.63         |2023-03-01 07:01:13|
|R-1aa09847|47.43         |2023-03-01 07:01:17|
|R-6c15250d|6.47          |2023-03-01 07:01:24|
|R-38ae50bd|5.8           |2023-03-01 07:01:33|
|R-8504417c|47.39         |2023-03-01 07:01:36|
|R-56c9ec2e|7.18          |2023-03-01 07:01:39|
|R-472dc5e0|49.33         |2023-03-01 07:01:40|
|R-a47862c0|47.87         |2023-03-01 07:01:46|
|R-d0997b94|7.05          |2023-03-01 07:01:48|
|R-ea97c3b7|47.74         |2023-03-01 07:01:58|
|R-ade2e5bc|6.41          |2023-03-01 07:02:05|
|R-65343ded|5.19          |2023-03-01 07:02:11|
|R-505c49b5|6.66          |2023-03-01 07

In [52]:
from pyspark.sql.functions import abs

anomalies = anomalies.withColumn(
    "z_score",
    abs((col("estimated_fare") - mean_fare) / std_fare)
)

anomalies.select("request_id", "estimated_fare", "z_score", "ts").show(truncate=False)


+----------+--------------+------------------+-------------------+
|request_id|estimated_fare|z_score           |ts                 |
+----------+--------------+------------------+-------------------+
|R-fe111a0d|47.56         |1.5252095216190649|2023-03-01 07:00:01|
|R-63f2cb5e|6.38          |1.6044232794542517|2023-03-01 07:00:25|
|R-c7205a20|7.55          |1.5155046204630962|2023-03-01 07:00:34|
|R-259d8079|47.81         |1.5442092350787133|2023-03-01 07:01:08|
|R-77534688|47.8          |1.5434492465403271|2023-03-01 07:01:12|
|R-ec25b018|48.63         |1.6065282952263609|2023-03-01 07:01:13|
|R-1aa09847|47.43         |1.5153296706200474|2023-03-01 07:01:17|
|R-6c15250d|6.47          |1.5975833826087782|2023-03-01 07:01:24|
|R-38ae50bd|5.8           |1.6485026146806363|2023-03-01 07:01:33|
|R-8504417c|47.39         |1.5122897164665037|2023-03-01 07:01:36|
|R-56c9ec2e|7.18          |1.5436241963833761|2023-03-01 07:01:39|
|R-472dc5e0|49.33         |1.6597274929133765|2023-03-01 07:01

# Heatmap

In [65]:
# Round to broader zones (1 decimal = ~11km block)
from pyspark.sql.functions import round

# Use higher precision for more dispersed zones
zone_demand_supply = rides.groupBy(
    window("ts", "30 seconds"),
    round("pickup_lat", 2).alias("lat_zone"),   # switch from 1 → 2
    round("pickup_lon", 2).alias("lon_zone")
).agg(count("request_id").alias("demand")) \
.join(
    rides.filter((col("is_active") == 1) | (col("is_completed") == 1)) \
         .groupBy(window("ts", "30 seconds"),
                  round("pickup_lat", 2).alias("lat_zone"),
                  round("pickup_lon", 2).alias("lon_zone")) \
         .agg(count("request_id").alias("supply")),
    on=["window", "lat_zone", "lon_zone"],
    how="outer"
).fillna(0) \
.withColumn("demand_supply_ratio", col("demand") / (col("supply") + 1))


# Convert to pandas
zone_pdf = zone_demand_supply.select(
    "lat_zone", "lon_zone", "demand_supply_ratio"
).toPandas()

# Plot updated heatmap
import plotly.express as px
fig = px.density_mapbox(
    zone_pdf,
    lat="lat_zone",
    lon="lon_zone",
    z="demand_supply_ratio",
    radius=25,
    center=dict(lat=zone_pdf["lat_zone"].mean(), lon=zone_pdf["lon_zone"].mean()),
    zoom=9,
    mapbox_style="carto-positron",
    color_continuous_scale="Turbo",  # or try "Viridis", "Inferno", "Plasma"
    title="Heatmap of Demand/Supply Ratio by Zone"
)

fig.show()



